In [72]:
import pandas as pd
import os

df = pd.read_excel(os.path.join('data','etunimitilasto-2022-08-04-dvv.xlsx'), sheet_name='Miehet ens')
print(df.shape)
df.head()

(6651, 2)


,Etunimi,Lukumäärä
0,Juha,44293
1,Timo,43711
2,Matti,40752
3,Kari,38967
4,Mikko,38053


In [73]:
names = list(df.Etunimi.str.lower())
chars = set(''.join(names))
print([e for e in chars])

['ø', 'ü', 'ö', 'h', 'å', 'w', 'ó', 'n', 'k', 'l', 'ä', 'v', 'f', 'ò', 'ú', '-', 'm', 'ã', 'o', 'u', 'ë', 'q', 'y', 'j', 't', 'r', 'ç', 'x', 'è', 'a', 'é', 'e', 'g', 'c', 'z', "'", '.', 'i', 'õ', 's', 'd', 'á', 'p', 'b', 'í']


In [74]:
c2i = {c:i for i, c in enumerate(chars, start=1)}
c2i['<s>'] = 0
c2i['<e>'] = len(c2i)

i2c = {i:c for c, i in c2i.items()}

In [75]:
import numpy as np

N = np.zeros(shape=(len(c2i),len(c2i)))

for name in names:
    name_chars = list(name)
    bigrams = [(a,b) for a,b in zip(['<s>'] + name_chars, name_chars + ['<e>'])]
    for a, b in bigrams:
        N[c2i[a], c2i[b]] += 1

Find the best bigram

In [76]:
idx = [(i, j) for i in range(len(c2i)) for j in range(len(c2i))]
bigrams = {(i2c[a],i2c[b]): N[a,b] for (a,b) in idx}

In [77]:
sorted(bigrams.items(), key=lambda kv:-kv[1])[:10]

[(('i', '<e>'), 1088.0),
 (('a', 'n'), 1044.0),
 (('n', '<e>'), 876.0),
 (('a', 'r'), 852.0),
 (('<s>', 'a'), 851.0),
 (('r', 'i'), 843.0),
 (('o', '<e>'), 709.0),
 (('e', 'r'), 703.0),
 (('<s>', 'j'), 675.0),
 (('k', 'a'), 634.0)]

In [ ]:
normalization = N.sum(axis=1, keepdims=True)
P = N / normalization

In [92]:
for _ in range(10):
    tokens = ['<s>']
    while True:
        recent = tokens[-1]
        next_token_idx = np.random.multinomial(n=1, pvals=P[c2i[recent]]).argmax()
        tokens.append(i2c[next_token_idx])

        if i2c[next_token_idx] == '<e>':
            break

    print(''.join(tokens[1:-1]))

velar
mog
ssarilkold
stoelan
veeko
jorarrisaditi
kas
urtele-vo-pe
a
sshdan
